In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree


from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import itertools

from QuadraticConstraintModel import train_COF_on_leaves, get_h_from_COF

from QuadraticConstraintModel import get_feature_bounds_from_COF, predict_from_COF, get_elevated_vertices

from Helping_Code.HelpingFunctions import load_dataset, normalized_root_mean_square_error
from Helping_Code.CustomHyperrectangle import CustomHyperrectangle
from Helping_Code.plot.CustomPlotClass import CustomPlotClass

from COFPipeline import COFPipeline




In [28]:
from Helping_Code import CustomHyperrectangle
from QuadraticConstraintModel import get_feature_bounds_from_COF, predict_from_COF, get_elevated_vertices
from QuadraticConstraintModel import train_COF_on_leaves, get_h_from_COF



In [2]:
data_Directory = "Dataset/"

X,y = load_dataset(data_Directory + "SteamGovernor/data_SteamGovernor_10000.csv",num_attributes = 3, num_classes = 3 )
print(f"Size of the Data Set\n Shape of X = {X.shape} \n Shape of y = {y.shape}")

X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.1)
print(f" Shape of X_Training = {X_train.shape} \n Shape of X_Testing = {X_test.shape}")

# X_train = X
# y_train = y


Size of the Data Set
 Shape of X = (10000, 3) 
 Shape of y = (10000, 3)
 Shape of X_Training = (9000, 3) 
 Shape of X_Testing = (1000, 3)


In [5]:
# 'max_depth': 5,
tree_params = { 'min_samples_leaf': 1000}
pipeline = COFPipeline(optimizer="gurobi", tree_params=tree_params, scale=True, poly_degree=3, n_jobs = -1)
#pipeline = COFPipeline(optimizer="gurobi", max_depth=5, scale=True,auto_tune_poly=True, max_poly_degree=3, n_jobs = 10)
pipeline.fit(X_train, y_train)



Optimization on Leaf ID = 3
Restricted license - for non-production use only - expires 2026-11-23
Optimization on Leaf ID = 10
Restricted license - for non-production use only - expires 2026-11-23
Optimization on Leaf ID = 5
Restricted license - for non-production use only - expires 2026-11-23
Optimization on Leaf ID = 9
Restricted license - for non-production use only - expires 2026-11-23
Optimization on Leaf ID = 4
Restricted license - for non-production use only - expires 2026-11-23
Optimization on Leaf ID = 8
Restricted license - for non-production use only - expires 2026-11-23


In [6]:
# Inspect some leaf bounds (original domain)
cof_model = pipeline.cof_model
print("Number of modeled leaves:", len(cof_model.leaf_models))
for lid, leaf in list(cof_model.leaf_models.items()):
    print("H = ", leaf.h)
    # print(f"leaf {lid}: samples={leaf.no_samples}, h={leaf.h:.6g}, bounds={leaf.bounds}")



y_pred = pipeline.predict(X_test)

print(y_test[:10])
print(y_pred[:10])



Number of modeled leaves: 6
H =  0.06313966764750206
H =  0.055615817070673645
H =  0.6340935063731664
H =  0.06742003777791006
H =  0.5512850631168359
H =  0.06322051425945645
[[-3.5649204e-01  1.5548590e+00  1.4460613e-01]
 [-1.0233556e+00  2.6182160e+00  5.4471445e-01]
 [-9.5747316e-01  2.4527330e+00  4.8113210e-01]
 [ 5.1089424e-01 -1.5426056e+00  6.9146720e-02]
 [ 7.7567184e-01 -2.9116590e+00  2.2426504e-01]
 [ 2.4515295e-01 -1.1335679e+00  1.6603774e-01]
 [-7.7777170e-01  2.8329868e+00  1.3420442e-01]
 [ 4.5827508e-02  5.7768920e-01  3.2671180e-01]
 [ 1.3372606e-01 -3.6143687e-01  9.5507760e-04]
 [-7.6486900e-01  2.9790533e+00  5.4508954e-01]]
[[-3.56491682e-01  1.55650777e+00  1.44095886e-01]
 [-1.02335964e+00  2.72432280e+00  5.48516082e-01]
 [-9.57476943e-01  2.50270585e+00  4.81443751e-01]
 [ 5.10892620e-01 -1.51847319e+00  6.83134123e-02]
 [ 7.75668825e-01 -2.87168246e+00  2.27265386e-01]
 [ 2.45152154e-01 -1.13491984e+00  1.66025322e-01]
 [-7.77771210e-01  2.83223687e+00  1

In [7]:
print("Number of modeled leaves:", len(cof_model.leaf_models))
for lid, leaf in list(cof_model.leaf_models.items()):
    print(f"leaf {lid}: samples={leaf.no_samples}, h={leaf.h:.6g}, bounds={leaf.bounds}")
    # print(f"leaf {lid}: samples={leaf.no_samples}, h={leaf.h:.6g}, bounds={leaf.bounds}")

Number of modeled leaves: 6
leaf 3: samples=1369, h=0.0631397, bounds={'feature_0': (-0.684021, 0.68399334), 'feature_1': (-1.0995957, -0.5525236), 'feature_2': (-1.097731, 1.0998595), 'feature_3': (3.528596416169094e-07, 0.467884728441), 'feature_4': (-0.73568368968032, 0.738069355699728), 'feature_5': (-0.7086963949346501, 0.7113378952565759), 'feature_6': (0.30528232855696, 1.20911070345849), 'feature_7': (-1.1734970586986, 1.1275678991722502), 'feature_8': (4.810249881193052e-07, 1.20969091974025), 'feature_9': (-0.3200429798329412, 0.3200041563281374), 'feature_10': (-0.5020941129086389, -3.017131305505655e-07), 'feature_11': (-0.47102832627756336, 0.4774074035022634), 'feature_12': (-0.811072681355215, 0.7930275112612493), 'feature_13': (-0.7239447985249271, 0.6647860694422838), 'feature_14': (-0.7262391269876358, 0.7566632078016925), 'feature_15': (-1.3295329303469308, -0.16867569119067435), 'feature_16': (-1.2016506711948967, 1.2703504649412303), 'feature_17': (-1.2721035521972

In [ ]:
COF_model_tree = train_COF_on_leaves(X_train_s, y_train, tree, optimizer="gurobi")


In [18]:
customHyperrectangle = CustomHyperrectangle.CustomHyperrectangle()

In [27]:
constraints_on_leaves

[[-0.684021,
  0.68399334,
  -1.0995957,
  -0.5525236,
  -1.097731,
  1.0998595,
  3.528596416169094e-07,
  0.467884728441,
  -0.73568368968032,
  0.738069355699728,
  -0.7086963949346501,
  0.7113378952565759,
  0.30528232855696,
  1.20911070345849,
  -1.1734970586986,
  1.1275678991722502,
  4.810249881193052e-07,
  1.20969091974025,
  -0.3200429798329412,
  0.3200041563281374,
  -0.5020941129086389,
  -3.017131305505655e-07,
  -0.47102832627756336,
  0.4774074035022634,
  -0.811072681355215,
  0.7930275112612493,
  -0.7239447985249271,
  0.6647860694422838,
  -0.7262391269876358,
  0.7566632078016925,
  -1.3295329303469308,
  -0.16867569119067435,
  -1.2016506711948967,
  1.2703504649412303,
  -1.2721035521972202,
  -5.118803359707019e-07,
  -1.3227805079096688,
  1.3304900501400514],
 [-0.682189,
  0.68229425,
  0.56458706,
  1.0999956,
  -1.0979806,
  1.0986881,
  2.2382451947411397e-08,
  0.46552544358306247,
  -0.7309007803369699,
  0.71691066502812,
  -0.7260280583352561,
  0.6

In [26]:

vertices_of_hyperrectangle = customHyperrectangle.getVerticesFromBounds(constraints_on_leaves)


ValueError: Input must be a non-empty 2D array.